# Interacting with the mockup

Make sure the mockup is running as described in the [Running the UR3e Mockup](0_running_the_mockup.ipynb) notebook.

The mockup listens for control messages with the `robotarm.ctrl` routing key. 

Specifically, it listens for 5 *types* of control messages defined in the [communication protocol](../communication/protocol.py):
- `LOAD_PROGRAM(JOINT_POSITIONS:[], MAX_VELOCITY: float, ACCELERATION: float)`: Load a program into the robot arm (a list of waypoints).
- `PLAY()`: Start executing the loaded program.
- `PAUSE()`: Pause the execution of the program.
- `STOP()`: Stop the execution of the program and clear the loaded program.
- `INJECT_FAULT(FAULT_TYPE: FaultTypes, ...)`: Inject a fault into the robot arm to simulate error conditions.

Additionally, the mockup publishes status messages to the `robotarm.pt.state` routing key, which can be subscribed to for monitoring the state of the robot arm. It publishes status messages of the following *types*:
- `ROBOT_MODE`: Current mode of the robot (e.g., RUNNING, Idle).
- `Q_ACTUAL`: Current joint positions (radians).
- `QD_ACTUAL`: Current joint velocities (radians/second).
- `Q_TARGET`: Target joint positions (radians).
- `TIMESTAMP`: Timestamp of the status message.
- `JOINT_MAX_SPEED`: Maximum joint speeds (radians/second).
- `JOINT_MAX_ACCELERATION`: Maximum joint accelerations (radians/second²).
- `TCP_POSE`: Current TCP pose (position and orientation).

You can send control messages to the mockup using any RabbitMQ client. Here's an example of how to send a control message to move the robot arm to a specific position using the communication module provided in this project.

> Make sure the credentials match those from your RabbitMQ server setup.

In [ ]:
import numpy as np
from communication import protocol
from communication.rabbitmq import Rabbitmq

# Initialize RabbitMQ connection (adjust parameters as needed)
try:
    rmq = Rabbitmq(
        ip="localhost",
        port=5672,
        username="ur3e",
        password="ur3e",
        vhost="/",
        exchange="UR3E_AMQP",
        type="topic",
    )
    rmq.connect_to_server()
    print("✓ Connected to RabbitMQ successfully")
except Exception as e:
    print(f"✗ Failed to connect to RabbitMQ: {e}")
    print("\nMake sure RabbitMQ is running. You can start it with:")
    print("  python -m startup.start_docker_rabbitmq")

def send_control_message(rmq, msg):
    """Send a control message to the UR3e Mockup via RabbitMQ."""
    try:
        rmq.send_message(
            routing_key=protocol.ROUTING_KEY_CTRL,
            message=msg
        )
        print(f"✓ Control message: {msg} sent successfully")
    except Exception as e:
        print(f"✗ Failed to send control message: {e}")

In [ ]:
# Construct control message for loading a program
position = [0.0, -np.pi/2, np.pi/2, -np.pi/2, -np.pi/2, 0.0]
vel = 60 # deg/s
acc = 80 # deg/s²

msg = {
    protocol.CtrlMsgKeys.TYPE: protocol.CtrlMsgFields.LOAD_PROGRAM,
    protocol.CtrlMsgKeys.JOINT_POSITIONS: [position],
    protocol.CtrlMsgKeys.MAX_VELOCITY: vel,
    protocol.CtrlMsgKeys.ACCELERATION: acc,
}

send_control_message(rmq, msg)

# send control message for starting program
msg_start = {
    protocol.CtrlMsgKeys.TYPE: protocol.CtrlMsgFields.PLAY,
}

send_control_message(rmq, msg_start)


You should see an output in the terminal where the mockup is running indicating that the control message was received and processed:

```bash
2026-01-09 15:24:12,033 - ur3e_mockup.robot_arm_mockup - INFO - Received control message: {'type': 'load_program', 'joint_positions': [[0.0, -1.5707963267948966, 1.5707963267948966, -1.5707963267948966, -1.5707963267948966, 0.0]], 'max_velocity': 60, 'acceleration': 80}
2026-01-09 15:24:12,033 - ur3e_mockup.robot_arm_mockup - INFO - Received control message: {'type': 'play'}
```

Remember to disconnect the RabbitMQ client after sending the messages to free up resources.

In [ ]:
rmq.close()

Try sending different control messages to see how the mockup responds!